In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install soundfile
!pip install datasets

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-082p3kox
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-082p3kox
  Resolved https://github.com/huggingface/datasets.git to commit ce4a0c573920607bc6c814605734091b06b860e7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-k9lm6gcg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-k9lm6gcg
  Resolved https://github.com/huggingface/transformers.git to commit 7e5d46ded433605a906fcab6be43ac85307cca9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/ta.tar.gz

--2024-08-05 12:50:48--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/ta.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.92.235.121, 52.92.201.17, 52.92.206.33, ...
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.92.235.121|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-08-05 12:50:48 ERROR 403: Forbidden.



In [ ]:
from datasets import load_dataset
common_voice_train = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7046it [00:00, 70454.36it/s]
Reading metadata...: 17421it [00:00, 90031.86it/s]
Reading metadata...: 26699it [00:00, 91279.52it/s]
Reading metadata...: 41710it [00:00, 87178.38it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 436it [00:00, 2097.18it/s]
Reading metadata...: 11758it [00:00, 29810.58it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11815it [00:00, 91872.75it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10219it [00:00, 102177.43it/s]
Reading metadata...: 20437it [00:00, 99518.36it/s] 
Reading metadata...: 30394it [00:00, 94520.67it/s]
Reading metadata...: 40015it [00:00, 95156.03it/s]
Reading metadata...: 49549it [00:00, 94619.83it/s]
Reading metadata...: 59192it [00:00, 95216.52it/s]
Reading metadata...: 68722it [00:00, 92965.59it/s]
Reading metadata...: 78032it [00:00, 88307.82it/s]
Reading metadata...: 87993it [00:00, 89164.75it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5575it [00:00, 63524.16it/s]


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
#show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=20)
show_random_elements(common_voice_train, num_examples=2)

,path,audio,sentence
0,/root/.cache/huggingface/datasets/downloads/extracted/632b3eb1435158228029bc5172c7ebd3ea99f1ec6eefc20631a0e783e206f951/ta_train_0/common_voice_ta_26476046.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/632b3eb1435158228029bc5172c7ebd3ea99f1ec6eefc20631a0e783e206f951/ta_train_0/common_voice_ta_26476046.mp3', 'array': [-2.842170943040401e-14, 6.821210263296962e-13, 1.3109513474773848e-12, 1.4779288903810084e-12, 1.2647660696529783e-12, 1.0800249583553523e-12, 5.968558980384842e-13, 1.4210854715202004e-13, 1.1368683772161603e-13, 3.979039320256561e-13, 1.3358203432289883e-12, 1.4495071809506044e-12, 3.872457909892546e-13, -1.0729195309977513e-12, -1.7053025658242404e-12, -1.7053025658242404e-12, -1.3784529073745944e-12, 0.0, 2.2311041902867146e-12, 3.794298208958935e-12, 3.112177182629239e-12, 3.126388037344441e-13, -2.3590018827235326e-12, -2.5295321393059567e-12, 1.4921397450962104e-13, 2.9345414986892138e-12, 2.8919089345436078e-12, -3.836930773104541e-13, -4.931166586175095e-12, -7.275957614183426e-12, -5.5422333389287814e-12, -4.121147867408581e-13, 5.6488147492927965e-12, 9.79127889877418e-12, 9.777068044058979e-12, 5.151434834260726e-12, -2.3163693185779266e-12, -9.066525308298878e-12, -1.0459189070388675e-11, -5.8264504332328215e-12, -4.405364961712621e-13, -8.952838470577262e-13, -8.79651906871004e-12, -2.0129675704083638e-11, -2.9693580927414587e-11, -3.373656909388956e-11, -3.19175796903437e-11, -2.651745489856694e-11, -2.0236257114447653e-11, -1.5830892152735032e-11, -1.5859313862165436e-11, -1.8673063095775433e-11, -1.864464138634503e-11, -1.5020873433968518e-11, -1.240607616637135e-11, -1.2448708730516955e-11, -8.952838470577262e-12, 2.7853275241795927e-12, 1.9241497284383513e-11, 3.11928260998684e-11, 3.278444182797102e-11, 2.7924329515371937e-11, 2.5835333872237243e-11, 2.6044943979286472e-11, 1.6711965145077556e-11, -8.121503469737945e-12, -3.878142251778627e-11, -6.178879630169831e-11, -7.226219622680219e-11, -7.436185001097328e-11, -7.628031539752556e-11, -8.388667538383743e-11, -9.43316535995109e-11, -1.0112444215337746e-10, -1.0672351891116705e-10, -1.1792167242674623e-10, -1.3207568372308742e-10, -1.3109513474773848e-10, -9.524114830128383e-11, -2.6687985155149363e-11, 4.4224179873708636e-11, 8.589040589868091e-11, 1.0017231488745892e-10, 1.127773430198431e-10, 1.311235564571689e-10, 1.3294254586071474e-10, 1.0148681894861511e-10, 5.3361759455583524e-11, 2.071942617476452e-11, 1.0828671292983927e-11, -3.723243935382925e-12, -4.5417891669785604e-11, -9.740119821799453e-11, -1.2570922081067692e-10, -1.2846612662542611e-10, -1.3747580851486418e-10, -1.7783463590603787e-10, -2.445261770844809e-10, -3.1444358228327474e-10, -3.610693966038525e-10, ...], 'sampling_rate': 48000}",தங்கம்மாளுக்குத் திருப்தி பிறந்தது.
1,/root/.cache/huggingface/datasets/downloads/extracted/632b3eb1435158228029bc5172c7ebd3ea99f1ec6eefc20631a0e783e206f951/ta_train_0/common_voice_ta_26356754.mp3,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/632b3eb1435158228029bc5172c7ebd3ea99f1ec6eefc20631a0e783e206f951/ta_train_0/common_voice_ta_26356754.mp3', 'array': [6.927637630131607e-24, -1.2407709188295415e-23, -1.1787323728880644e-23, -1.2252612823441722e-23, 2.740035779081904e-24, 1.34416849539867e-23, 6.203854594147708e-25, -1.1787323728880644e-23, -9.098986738083304e-24, -9.822769774067204e-24, -6.203854594147708e-25, 5.790264287871194e-24, 4.1359030627651384e-25, 4.1359030627651384e-24, -1.4889251025954498e-23, -1.6285118309637732e-24, 5.240964662347699e-24, -2.481541837659083e-24, -2.714186384939622e-24, 3.360421238496675e-25, 4.652890945610781e-24, 1.58973773975035e-23, 1.6172027210265248e-24, -4.652890945610781e-25, 6.2038545941477076e-24, 1.0430230536410833e-23, 1.2149215246872594e-24, 8.969739767371894e-24, 7.625571271973224e-24, 1.783608195817466e-23, 5.7644148937289116e-24, 2.9403685836845906e-24, 2.3781442610899546e-24, 1.6543612251060553e-24, 1.1580528575742387e-23, -2.0

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters, remove_columns=["sentence"])
common_voice_test = common_voice_test.map(remove_special_characters, remove_columns=["sentence"])

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,audio,sentence
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f06aa214dced4b4ddee7f1a3baccd6fa06e293893bbb2e6913f33f818b262c56/ta_dev_0/common_voice_ta_19093842.mp3', 'array': [0.0, 3.1644405520427077e-15, -4.3891506674011236e-15, 7.516266289106597e-15, 1.441249090829589e-14, 1.0439964830946582e-14, 5.770223064822683e-15, -1.5918002186548e-15, -8.06575794957238e-16, 1.7082794461767067e-14, 1.8182399081365444e-14, 2.186661293116128e-15, -2.0665213586027777e-14, -3.390980530225407e-14, 1.2527188691219791e-14, 6.445840091068647e-14, 0.0, -7.735008651383463e-14, -2.192257978611816e-14, 5.934215927894462e-14, 5.510723510003014e-14, -6.559983879348384e-15, -9.091199201869543e-14, -9.395536784565298e-14, 5.242742614282488e-15, 1.11426015305836e-14, -9.232356903716293e-14, -1.7747939619686126e-13, -2.738373221754242e-13, -1.5784159461233738e-13, 1.0533961940575876e-13, -8.22754543531104e-14, -3.573892755088376e-13, -9.801981139733948e-14, -2.5957447996605154e-13, -2.391094050188669e-13, 5.528702495816162e-13, 1.0171139104564464e-13, -1.3907288948927787e-12, -4.77178518053234e-13, 1.1791071591327196e-12, -2.9530707306574255e-13, -1.1709036518148253e-12, 1.2205893847733185e-14, -6.795334152147336e-13, -7.811410481124714e-13, 2.024575385811689e-12, 5.519575681928179e-13, -3.0950569798043892e-12, 1.7954041031664758e-13, 3.5580006822744092e-12, 4.825539924244171e-14, -1.1532077376363858e-12, 8.818801808599397e-13, -6.119859393555194e-13, -2.074052193011544e-13, 2.7436850534579005e-12, -2.9585020414404917e-13, -3.4789783900923688e-12, -3.34215673799404e-13, 6.641908160617827e-13, -7.277129203051014e-13, 7.076469401776087e-13, -1.978548835185334e-12, -5.519901809941663e-12, 7.391790088004391e-13, 4.698783029333242e-12, -2.5926546066284972e-12, -4.765636670012174e-12, 1.1327117629272854e-12, 1.7594255045938478e-12, 9.88845615633449e-13, 3.895269523601641e-12, 5.343673637261959e-13, -4.070016459273296e-12, 3.6155566388029747e-12, 1.0656339526071701e-11, 1.3982572695178663e-12, -7.688556388774082e-12, 1.7000825660445917e-12, 1.2321336659293358e-11, 2.9385994505803925e-12, -9.046187410266526e-12, 4.083836675945751e-13, 1.2271868417290666e-11, 2.7875904709540045e-12, -4.42925297133856e-12, 5.602464239057303e-12, 5.737428761254382e-12, 2.1703938559575198e-13, 1.271203281527633e-11, 1.1903316006478537e-11, -1.2637730271991554e-11, -2.529624730171487e-12, 2.8729865708676527e-11, 7.837256017773075e-12, -2.393393989941206e-11, -2.4840685064475565e-12, 1.52803047992478e-11, -2.9055347537665366e-12, ...], 'sampling_rate': 48000}",உலகிலேனும் எண்ணினால்
1,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b8a0e79ad40d8677943396d282fa6aa0bb270938d56eb8381969205bc9514cf6/ta_train_0/common_voice_ta_25966151.mp3', 'array': [-2.842170943040401e-14, 9.947598300641403e-14, 7.673861546209082e-13, 1.1368683772161603e-12, 9.379164112033322e-13, -1.1368683772161603e-13, -7.389644451905042e-13, -1.7053025658242404e-13, 7.389644451905042e-13, 2.2737367544323206e-13, -1.3358203432289883e-12, -1.5916157281026244e-12, -8.526512829121202e-14, 1.3073986337985843e-12, 6.536993168992922e-13, -1.4566126083082054e-12, -3.26139115713886e-12, -3.183231456205249e-12, -2.5579538487363607e-12, -3.5456082514429e-12, -4.035882739117369e-12, -1.8189894035458565e-12, 1.2221335055073723e-12, 1.9895196601282805e-12, -1.4210854715202004e-13, -2.5437429940211587e-12, -1.6200374375330284e-12, 3.069544618483633e-12, 6.764366844436154e-12, 6.991740519879386e-12, 6.650680006714538e-12, 8.697043085703626e-12, 1.1027623258996755e-11, 7.275957614183426e-12, -2.2026824808563106e-12, -7.645439836778678e-12, -1.9895196601282805e-13, 1.1368683772161603e-11, 9.492850949754938e-12, -4.810374321095878e-12, -1.0032863428932615e-11, 2.6716406864579767e-12, 1.4495071809506044e-11, 3.979039320256561e-12, -2.0161650127192843e-11, -3.0183855415089056e-11, -1.830358087318018e-11, -6.366462912410498e-12, -8.128608897095546e-12, -6.508571459562518e-12, 1.1709744285326451e-11, 2.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ே': 0,
 '\\': 1,
 '◯': 2,
 '்': 3,
 'h': 4,
 'ஓ': 5,
 'ô': 6,
 'த': 7,
 '.': 8,
 'w': 9,
 'ல': 10,
 ':': 11,
 'இ': 12,
 'n': 13,
 'ட': 14,
 'க': 15,
 'f': 16,
 'ம': 17,
 'ச': 18,
 "'": 19,
 'T': 20,
 'ய': 21,
 'J': 22,
 '(': 23,
 'ந': 24,
 'H': 25,
 'M': 26,
 'm': 27,
 'K': 28,
 'j': 29,
 'L': 30,
 'எ': 31,
 '…': 32,
 '_': 33,
 'ொ': 34,
 'ஒ': 35,
 '"': 36,
 'ெ': 37,
 'ூ': 38,
 'é': 39,
 '-': 40,
 'ஷ': 41,
 'c': 42,
 'ண': 43,
 'W': 44,
 ')': 45,
 'ள': 46,
 'X': 47,
 'ோ': 48,
 'ஹ': 49,
 '●': 50,
 'ை': 51,
 'o': 52,
 'E': 53,
 '‚': 54,
 'g': 55,
 'உ': 56,
 'v': 57,
 'ற': 58,
 'U': 59,
 'p': 60,
 'F': 61,
 'ங': 62,
 'ഥ': 63,
 'R': 64,
 'I': 65,
 'ஃ': 66,
 ',': 67,
 'ஔ': 68,
 '—': 69,
 'd': 70,
 '“': 71,
 'S': 72,
 'ர': 73,
 '!': 74,
 '”': 75,
 '&': 76,
 'ப': 77,
 '·': 78,
 'G': 79,
 'a': 80,
 'q': 81,
 'y': 82,
 'அ': 83,
 'ஞ': 84,
 'ஜ': 85,
 'ழ': 86,
 '’': 87,
 'V': 88,
 'O': 89,
 '¾': 90,
 'ஈ': 91,
 'ௗ': 92,
 'z': 93,
 '–': 94,
 'ி': 95,
 'r': 96,
 'ஐ': 97,
 'ஆ': 98,
 'ஊ': 99,
 'ா': 100

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

136

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
 processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-tamil-demo")

[]

In [ ]:
common_voice_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b8a0e79ad40d8677943396d282fa6aa0bb270938d56eb8381969205bc9514cf6/ta_train_0/common_voice_ta_26650298.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/b8a0e79ad40d8677943396d282fa6aa0bb270938d56eb8381969205bc9514cf6/ta_train_0/common_voice_ta_26650298.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 48000},
 'sentence': 'அவரைப் பொதுமக்கள் விடாமல் பின்னாலேயே துரத்திக் கொண்டே ஓடினார்கள்.'}

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/53468 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
import librosa
import numpy as np

def resample(batch):
    # Pass a tuple to librosa.resample
    batch["speech"] = librosa.resample((np.asarray(batch["speech"]), 48_000, 16_000))
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

Map (num_proc=4):   0%|          | 0/53468 [00:00<?, ? examples/s]

TypeError: resample() missing 2 required keyword-only arguments: 'orig_sr' and 'target_sr'

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train))

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
ipd.Audio(data=np.asarray(common_voice_train[200]["speech"]), autoplay=True, rate=16000)

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/53468 [00:00<?, ? examples/s]

ValueError: The model corresponding to this feature extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": true,
  "sampling_rate": 16000
}
 was trained using a sampling rate of 16000. Please make sure that the provided `raw_speech` input was sampled with 16000 and not 32000.

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

!rm -rf ~/.cache/huggingface/datasets/common_voice

#test_dataset = load_dataset("common_voice", "ta", split="test[:2%]")
test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="test[:2%]")

processor = Wav2Vec2Processor.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Prediction: ['மூஙகஅகற்கவா நாக்கப்பகும் ்்', 'எங்க நாளைல் அபே மறைந்தே வழுகி ரதே']
Reference: ['உங்கள் நோக்கு புதிது.', 'இந்த நாளில் அது மறைந்து வருகிறது.']


In [ ]:
from datasets import load_dataset



In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [ ]:
test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="test")


In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

#test_dataset = load_dataset("common_voice", "ta", split="test")
test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", trust_remote_code=True, split="test")
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


The repository for wer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

Parameter 'function'=<function evaluate at 0x7b02e7de8dc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

WER: 92.175006


In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.4 MB/s eta 0:00:00
